In [1]:
import itertools
import copy
import numpy as np

In [2]:
numbers= """5616185650518293
3847439647293047 
5855462940810587 
9742855507068353 
4296849643607543 
3174248439465858 
4513559094146117 
7890971548908067 
8157356344118483 
2615250744386899 
8690095851526254 
6375711915077050 
6913859173121360 
6442889055042768
2321386104303845 
2326509471271448 
5251583379644322 
1748270476758276 
4895722652190306 
3041631117224635 
1841236454324589 
2659862637316867"""
numbers = numbers.split('\n')
Ncor = [2,1,3,3,3,1,2,3,1,2,3,1,1,2,0,2,2,3,1,3,3,2]
L = 16
N = len(Ncor)

In [ ]:
numbers = ['90342','70794','39458','34109','51545','12531']
Ncor = [2,0,2,1,2,1]
L = len(numbers[0])
N = len(Ncor)

In [54]:
def remove_impossible(pos_dig, ncor):
## Checks for numbers that have 0 remaining degrees of freedom
## The digits of those numbers are impossible 
## They are removed from the list of possible digits. 
  temp = copy.deepcopy(pos_dig)
  for a in range(len(ncor)):
    if ncor[a] == 0:
      num = numbers[a]
      for i in range(L):
        if num[i] in temp[i]:
          temp[i].remove(numbers[a][i])
  return temp

In [55]:
def guess1digit(prev_guess, ind, dig, pos_dig, dof):
  ### prev_guess will have a mix of '.' and digits.
  ### make the new guess that the '.' at position ind is the digit dig
  ### pos_dig is list of possible digits (for each position) from the prev_guess
  ### dof is the remaining degree of freedom from prev_guess
  ### If the guess of that 1 digit is invalid (inconsistent), the function will return False
  ### If the guess of that 1 digit is valid, the function will return
  ### new_guess, new_pos_dig, new_dof, True
  
  ### If the digit being guessed is not in pos_dig
  if not (dig in pos_dig[ind]):
    return prev_guess,pos_dig,dof,False
  ### If the position being guessed is not available, i.e. not '.'
  if prev_guess[ind] != '.':
    return prev_guess,pos_dig,dof,False
  
  ###  

  new_dof = dof.copy()
  new_pos_dig = copy.deepcopy(pos_dig)
  for i in range(N):
    if numbers[i][ind] == dig:
      ### If a number with 0 dof has its digit match the guess, then there is inconsistency.
      if dof[i] <= 0:
        return prev_guess,pos_dig,dof,False
      new_dof[i] -= 1
      
      if dof[i] == 1:
        for a in range(L):
#          if prev_guess[a] == numbers[i][a]:
#            return prev_guess, [], [], False
          if numbers[i][a] in new_pos_dig[a]:
            new_pos_dig[a].remove(numbers[i][a])
  new_pos_dig[ind] = set()
  newguess = prev_guess[:ind]+dig+prev_guess[ind+1:]
  
  ### If there is no more possible digits for unguessed position, return False
  for i in range(L):
    if newguess[i]=='.' and len(new_pos_dig[i]) == 0:
      return prev_guess,pos_dig,dof,False
  
  ### If the maximum number of dof left is more than the number of unguessed positions, return False
  unguessed = 0
  for i in range(L):
    if newguess[i]=='.':
      unguessed += 1
  if unguessed < max(new_dof):
    return prev_guess,pos_dig,dof,False

  return newguess, new_pos_dig, new_dof, True

In [ ]:
def forced_guess(prev_guess, pos_dig, dof):
## if the there is only one possible digit for a positiion, then 
## we must guess that digit for that position  
  for i in range(L):
    if len(pos_dig[i]) == 1 and prev_guess[i] == '.':
      dig = pos_dig[i].pop()
      pos_dig[i].add(dig)
      return guess1digit(prev_guess, i, dig, pos_dig, dof) 
### If the return value is False, then prev_guess should be rejected. 
  unguessed = 0
  first_unguessed = 0
  for i in range(L):
    if prev_guess[i] =='.':
      if unguessed == 0:
        first_unguessed = i 
      unguessed+=1   
  for i in range(L):
    if len(pos_dig[i]) == unguessed:
      return guess1digit(prev_guess, first_unguessed, numbers[i][first_unguessed], pos_dig, dof)

In [ ]:
def make_guess(prev_guess,pos_dig, dof):
  print('Making a guess with', prev_guess)
  #print(pos_dig)
  forcing = True
  while forcing:
    forcing = False
    for i in range(L):
      if len(pos_dig[i]) == 1 and prev_guess[i] == '.':
        dig = pos_dig[i].pop()
        pos_dig[i].add(dig)
        prev_guess, pos_dig, dof, isvalid = guess1digit(prev_guess, i, dig, pos_dig, dof) 
        if not isvalid:
          print('forcing position',i,'with',dig, 'was invalid')
          return False
        if isvalid:
          print('forcing position',i,'with',dig, 'was valid')
          forcing = True
          
          fullyguessed = True
          for i in range(L):
            fullyguessed *= (prev_guess[i] != '.')
          if fullyguessed:
            return True
          break

    unguessed = 0
    first_unguessed = 0
    for i in range(L):
      if prev_guess[i] =='.':
        if unguessed == 0:
          first_unguessed = i 
        unguessed+=1   
    for i in range(N):
      if dof[i] == unguessed:
        prev_guess, pos_dig, dof, isvalid = guess1digit(prev_guess, first_unguessed, numbers[i][first_unguessed], pos_dig, dof)
        if not isvalid:
          return False
        if isvalid: 
          forcing = True
          print('forcing position',first_unguessed,'with',numbers[i][first_unguessed], 'was valid')
          fullyguessed = True
          for i in range(L):
            fullyguessed *= (prev_guess[i] != '.')
          if fullyguessed:
            return True 
          break

  if 1 in dof:
    ind = dof.index(1)
  elif 2 in dof:
    ind = dof.index(2)
  else: 
    return True
  for a in range(L):
    if prev_guess[a] == '.' and numbers[ind][a] in pos_dig[a]:
      new_guess, new_pos_dig, new_dof, isvalid = guess1digit(prev_guess, a, numbers[ind][a], pos_dig, dof)
      if isvalid:
        if make_guess(new_guess,new_pos_dig, new_dof):
          return True
  return False



In [ ]:
ini_pos_dig = []
for a in range(L):
  temp = set()
  for i in range(N):
    temp.add(numbers[i][a])
  ini_pos_dig.append(temp)
pos_dig = remove_impossible(ini_pos_dig, Ncor)
make_guess('.'*L,pos_dig, Ncor)

In [ ]:
ini_pos_dig = []
for a in range(L):
  temp = set()
  for i in range(N):
    temp.add(numbers[i][a])
  ini_pos_dig.append(temp)
pos_dig = remove_impossible(ini_pos_dig, Ncor)
guess, pos_dig, dof, isvalid = guess1digit('.'*L, 0, '3', pos_dig, Ncor)
print(guess, isvalid)

3............... True


In [ ]:
Ncor

[2, 0, 2, 1, 2, 1]

In [ ]:
guess, pos_dig, dof, isvalid = guess1digit(guess, 4, '8', pos_dig, dof)
print(guess, isvalid)

32558........... True


In [ ]:
guess, pos_dig, dof, isvalid = forced_guess(guess, pos_dig, dof)
print(guess, isvalid)

TypeError: ignored

In [ ]:
dof

[2, 0, 1, 2, 1, 0, 1, 3, 0, 1, 3, 0, 0, 0, 0, 2, 0, 3, 0, 2, 3, 0]

In [ ]:
# Look for frequncies of digit for each position.
# Start by guessing the highest frequency digit
# If not hard reject, then guess the next highest frequency digit.
# Keep going.....
# If there is a reject, then we have to reguess the previous digit by choosing the next highest frequency digit.


In [ ]:
def check(guess):
#print('check', guess)
  for i in range(N):
    count = 0
    for j in range(L):
      if guess[j] == numbers[i][j]:
        count += 1
    if count != Ncor[i]:
      return(False)
  code = guess
  return(True)  


In [ ]:
def makeguess(guess, k, pos_dig, ncor):

### guess will contain some number of '.' and some digits
### To make guess for some '.''s, take d digits from the kth number 
### And these digits have to be in pos_dig.
### d is the remaining number of correct digits for the kth number.
### Pass the new guess to makeguess
### ncor and pos_dig has to be updated before passing to the next makeguess

#print('makeguess',guess)

### If all numbers have been used. No clue to guess anymore
  if k >= N:
    return check(guess)
  if -1 in ncor or -2 in ncor or -3 in ncor: #This shouldn't happen
    return False
### Find position of unguessed indices 
  ind_of_dots= []
  for i in range(L):
    if guess[i] == '.':
      ind_of_dots.append(i)
  if len(ind_of_dots) == 0:
    return(check(guess))

### If there are less '.''s than remaining correct digits for the kth number
### Then makeguess using the k+1 number
  if len(ind_of_dots) < ncor[k]:
    return(makeguess(guess,k+1,pos_dig,ncor))

  ref = numbers[k]
  list_ind_to_guess = list(itertools.combinations(ind_of_dots, ncor[k]))
  for ind_to_guess in list_ind_to_guess:
    next_guess = guess
    valid_guess= True
    for i in ind_to_guess:
      if not (ref[i] in pos_dig[i]):
        valid_guess = False
        break
      next_guess = next_guess[:i] + ref[i] + next_guess[i+1:]
    if valid_guess:
      new_ncor = ncor.copy()
      for i in ind_to_guess:
        for a in range(N):
          if ref[i] == numbers[a][i]: 
            new_ncor[a] -= 1
      new_pos_dig = remove_impossible(pos_dig, new_ncor)
      if makeguess(next_guess,k+1,new_pos_dig,new_ncor):
        return(True)
  return(False)
            


In [ ]:
code = ''
ini_guess = '.'*L
pos_dig = []
for i in range(L):
  pos_dig.append({'0','1','2','3','4','5','6','7','8','9'})

ini_pos_dig = remove_impossible(ini_pos_dig, Ncor)
makeguess(ini_guess,0, ini_pos_dig, Ncor)

In [ ]:
code = ''
ini_guess = '.'*L
ini_pos_dig = []
for i in range(L):
  ini_pos_dig.append({'0','1','2','3','4','5','6','7','8','9'})
remove_impossible(ini_pos_dig,Ncor)
ini_pos_dig

[{'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'},
 {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'},
 {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'},
 {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'},
 {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}]

In [ ]:
Allfreq = np.zeros((10,L),dtype=int)
for a in range(L):
  for i in range(N):
    Allfreq[int(numbers[i][a]),a] += 1

Allfreq

array([[0, 2, 0, 1, 0],
       [1, 1, 1, 0, 1],
       [0, 1, 0, 0, 1],
       [2, 0, 1, 1, 0],
       [0, 1, 1, 2, 1],
       [1, 0, 2, 1, 1],
       [0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 1, 0, 1, 1]])

In [ ]:
np.where(Allfreq==6)

(array([4, 4, 9]), array([2, 8, 6]))

In [ ]:
pos_dig = []
for i in range(L):
  pos_dig.append({'0','1','2','3','4','5','6','7','8','9'})
guess = '.'*16
places = np.where(Allfreq==6)
dof=Ncor
for i in range(places[0].size):
  guess, pos_dig, dof, isvalid = guess1digit(guess, places[1][i], str(places[0][i]), pos_dig, dof)


In [ ]:
print(guess)
print(pos_dig)
print(dof, isvalid)

.24025155.6..2..
[{'0'}, set(), set(), set(), set(), set(), set(), set(), set(), set(), set(), {'3'}, {'9'}, set(), {'1', '3'}, {'5', '1'}]
[0, 0, 3, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 0, 2] False


In [ ]:
places = np.where(Allfreq==2)
for i in range(places[0].size):
  guess, pos_dig, dof, isvalid = guess1digit(guess, places[1][i], str(places[0][i]), pos_dig, dof)

In [ ]:
highfreq = []
for i in range(L):
  temp = []
  for k in range(10):
    count = 0
    for n in range(N): 
      if int(numbers[n][i]) == k:
        count += 1
    if count >= 5:
      temp.append(k)
  highfreq.append(temp)

highfreq

[[], [8], [4], [], [8], [5], [9], [6], [4, 5], [4], [], [], [8], [], [], [7]]

In [ ]:
powerset=[]
for i in range(N):
  for el in itertools.combinations(range(N),i):
    powerset.append(el)
len(powerset)

4194303

In [ ]:
NotableSets=[]
for el in powerset:
  TotCor=0
  for i in el:
    TotCor += Ncor[i]
  if TotCor > L:
    freq = np.zeros((10,L),dtype=int)
    for a in range(L):
      for i in el:
        freq[int(numbers[i][a]),a] += 1
    if np.sum(np.amax(freq,axis=0)) == TotCor:
      NotableSets.append(el)

KeyboardInterrupt: ignored

In [ ]:
  TotCor=0
  for i in [2,3,4,7,10,17,19,20]:
    TotCor += Ncor[i]
  if TotCor > L:
    freq = np.zeros((10,L),dtype=int)
    for a in range(L):
      for i in [2,3,4,7,10,17,19,20]:
        freq[int(numbers[i][a]),a] += 1

  print(np.sum(np.amax(freq,axis=0)))

38


In [ ]:
code

''

In [57]:
freq = np.zeros((10,L),dtype=int)
for a in range(L):
  for i in range(N):
    freq[int(numbers[i][a]),a] += 1
print(freq)
np.amax(freq, axis=0)

[[0 1 0 2 1 1 2 2 2 2 3 3 2 3 1 2]
 [2 2 4 4 1 1 3 3 2 2 4 4 2 1 1 0]
 [4 2 2 2 4 1 3 0 0 1 3 4 1 3 1 1]
 [3 3 0 2 2 3 1 2 2 2 4 0 2 4 1 4]
 [3 1 6 1 2 2 0 4 6 5 1 3 3 2 4 1]
 [3 1 4 4 3 5 3 2 5 2 2 1 1 3 4 2]
 [3 4 0 3 1 2 3 5 0 1 2 2 4 1 4 2]
 [1 2 2 2 2 2 0 1 4 4 1 2 2 1 1 5]
 [2 5 0 1 5 4 1 1 0 1 1 1 5 4 3 3]
 [1 1 4 1 1 1 6 2 1 2 1 2 0 0 2 2]]


array([4, 5, 6, 4, 5, 5, 6, 5, 6, 5, 4, 4, 5, 4, 4, 5])

In [67]:
ini_pos_dig = []
for a in range(L):
  temp = set()
  for i in range(N):
    temp.add(numbers[i][a])
  ini_pos_dig.append(temp)
pos_dig = remove_impossible(ini_pos_dig, Ncor)


In [68]:
ini_guess = '.'*L
guess, pos_dig, dof, isvalid = guess1digit(ini_guess,2,'4',pos_dig, Ncor)
print(guess, isvalid)

..4............. True


In [81]:
guess, pos_dig, dof, isvalid = guess1digit(guess,0,'2',pos_dig, dof)
print(guess, isvalid)

..4.8.......8... False


In [80]:
pos_dig

[{'1', '4', '5', '7'},
 {'0', '2', '5', '6'},
 set(),
 {'0', '4', '5', '6', '8', '9'},
 set(),
 {'0', '1', '2', '4', '6', '7', '9'},
 {'0', '1', '2', '3', '8'},
 {'4', '7', '8', '9'},
 {'1', '3', '9'},
 {'0', '1', '2', '6', '8', '9'},
 {'4', '5', '6', '7', '8', '9'},
 {'5', '7', '8'},
 set(),
 {'1', '2', '5', '6'},
 {'0', '1', '2', '3', '7', '9'},
 {'2', '4', '6', '9'}]